In [1]:
import numpy as np
from pyscf import gto, scf, ao2mo, mcscf

PSPCz_mol = gto.M(atom=[
                  ['C', (  -0.2316640,    1.1348450,    0.6956120)],
                  ['C', (  -0.8886300,    0.3253780,   -0.2344140)],
                  ['C', (  -0.1842470,   -0.1935670,   -1.3239330)],
                  ['C', (   1.1662930,    0.0801450,   -1.4737160)],
                  ['C', (   1.8089230,    0.8832220,   -0.5383540)],
                  ['C', (   1.1155860,    1.4218050,    0.5392780)],
                  ['S', (   3.5450920,    1.2449890,   -0.7349240)],
                  ['O', (   3.8606900,    1.0881590,   -2.1541690)],
                  ['C', (   4.3889120,   -0.0620730,    0.1436780)],
                  ['O', (   3.8088290,    2.4916780,   -0.0174650)],
                  ['C', (   4.6830900,    0.1064460,    1.4918230)],
                  ['C', (   5.3364470,   -0.9144080,    2.1705280)],
                  ['C', (   5.6895490,   -2.0818670,    1.5007820)],
                  ['C', (   5.4000540,   -2.2323130,    0.1481350)],
                  ['C', (   4.7467230,   -1.2180160,   -0.5404770)],
                  ['N', (  -2.2589180,    0.0399120,   -0.0793330)],
                  ['C', (  -2.8394600,   -1.2343990,   -0.1494160)],
                  ['C', (  -4.2635450,   -1.0769890,    0.0660760)],
                  ['C', (  -4.5212550,    0.2638010,    0.2662190)],
                  ['C', (  -3.2669630,    0.9823890,    0.1722720)],
                  ['C', (  -2.2678900,   -2.4598950,   -0.3287380)],
                  ['C', (  -3.1299420,   -3.6058560,   -0.3236210)],
                  ['C', (  -4.5179520,   -3.4797390,   -0.1395160)],
                  ['C', (  -5.1056310,   -2.2512990,    0.0536940)],
                  ['C', (  -5.7352450,    1.0074800,    0.5140960)],
                  ['C', (  -5.6563790,    2.3761270,    0.6274610)],
                  ['C', (  -4.4287740,    3.0501460,    0.5083650)],
                  ['C', (  -3.2040560,    2.3409470,    0.2746950)],
                  ['H', (  -0.7813570,    1.5286610,    1.5426490)],
                  ['H', (  -0.7079140,   -0.7911480,   -2.0611600)],
                  ['H', (   1.7161320,   -0.2933710,   -2.3302930)],
                  ['H', (   1.6308220,    2.0660550,    1.2427990)],
                  ['H', (   4.4214900,    1.0345500,    1.9875450)],
                  ['H', (   5.5773000,   -0.7951290,    3.2218590)],
                  ['H', (   6.2017810,   -2.8762260,    2.0345740)],
                  ['H', (   5.6906680,   -3.1381740,   -0.3739110)],
                  ['H', (   4.5337010,   -1.3031330,   -1.6001680)],
                  ['H', (  -1.1998460,   -2.5827750,   -0.4596910)],
                  ['H', (  -2.6937370,   -4.5881470,   -0.4657540)],
                  ['H', (  -5.1332290,   -4.3740010,   -0.1501080)],
                  ['H', (  -6.1752900,   -2.1516170,    0.1987120)],
                  ['H', (  -6.6812260,    0.4853900,    0.6017680)],
                  ['H', (  -6.5574610,    2.9529350,    0.8109620)],
                  ['H', (  -4.3980410,    4.1305040,    0.5929440)],
                  ['H', (  -2.2726630,    2.8838620,    0.1712760)]],
            charge=0,spin=0,basis='631g*',verbose=0)


In [1]:
import qiskit
from qiskit_nature.drivers import PySCFDriver, UnitsType
from qiskit_nature.transformers import FreezeCoreTransformer
from qiskit_nature.algorithms import GroundStateEigensolver
from qiskit_nature.mappers.second_quantization import ParityMapper
from qiskit_nature.problems.second_quantization.electronic import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.circuit.library import HartreeFock, UCCSD
from qiskit.circuit.library import TwoLocal

ImportError: cannot import name 'PySCFDriver' from 'qiskit_nature.drivers' (/home/taamangtchu/mambaforge/envs/qiskit-env/lib/python3.10/site-packages/qiskit_nature/drivers/__init__.py)

In [3]:
# Utilisation de la fonction PySCFDriver pour générer l'Hamiltonien
# à partir de la molécule C18H13NO2S
molecule = 'C18H13NO2S'
driver = PySCFDriver(molecule=molecule, unit=UnitsType.ANGSTROM, basis='sto3g')
problem = ElectronicStructureProblem(driver, [FreezeCoreTransformer()])

# Utilisation de la fonction ParityMapper pour mapper l'Hamiltonien
# sur des qubits, et utilisation de la fonction QubitConverter pour
# convertir l'Hamiltonien en un format supporté par les algos de Qiskit
mapper = ParityMapper()
converter = QubitConverter(mapper=mapper)
qubit_op = converter.convert(problem.second_q_ops())

# Utilisation de l'algorithme GroundStateEigensolver pour calculer
# l'état fondamental de l'Hamiltonien
solver = GroundStateEigensolver(converter)
result = solver.solve(problem)

# Utilisation de la fonction HartreeFock pour générer un état initial,
# et utilisation de la fonction UCCSD pour construire un circuit quantique
# variational pour optimiser l'énergie de l'état fondamental
num_particles = (problem.molecule_data_transformed.num_alpha, problem.molecule_data_transformed.num_beta)
num_spin_orbitals = 2 * problem.molecule_data_transformed.num_molecular_orbitals
init_state = HartreeFock(num_spin_orbitals, num_particles, converter)
var_form = UCCSD(num_orbitals=num_spin_orbitals, num_particles=num_particles, converter=converter, initial_state=init_state)

# Utilisation de la fonction TwoLocal pour créer un circuit quantique
# pour optimiser l'énergie de l'état fondamental
ansatz = TwoLocal(num_spin_orbitals, ['ry', 'rz'], 'cz', reps=3)

# Utilisation de la méthode VQE pour calculer l'énergie de l'état fondamental
# et le circuit quantique optimisé qui le réalise
from qiskit.algorithms import VQE
from qiskit.providers.aer import QasmSimulator

backend = QasmSimulator()
vqe = VQE(ansatz=ansatz, optimizer=qiskit.optimization.COBYLA(), quantum_instance=backend)
result = vqe.compute_minimum_eigenvalue(qubit_op)

# Affichage du résultat
print(result)

(430, 430)
2523.279371819741


In [2]:
import latex2sympy2
import flask

$  \begin{bmatrix} 1 & 1 & 1 & 1 \\
    0 & 1 & 0 & 0 \\
    0 & 0 & 1 & 1 \\
    0 & 0 & 0 & 1\end{bmatrix} \begin{bmatrix} 0 \\ 1 \\ 1\\ 0\end{bmatrix} = \begin{bmatrix}2\\1\\1\\0\end{bmatrix}$

In [ ]:
import netket as nk
netket
nk.__version__


In [ ]:
hi = nk.hilbert.Spin(1/2)
hi

In [ ]:
nk.operator.spin.sigmaz(hi,0)

In [ ]:
from qiskit_nature.second_q.operators import FermionicOp
from qiskit_nature.second_q.operators.commutators import anti_commutator

In [ ]:
op1 = FermionicOp({"+_0 -_1": 1}, num_spin_orbitals=2)
op2 = FermionicOp({"-_1 +_2": 1}, num_spin_orbitals=3)

In [ ]:
anti_commutator(FermionicOp({"+_0": 1}, num_spin_orbitals=1), FermionicOp({"-_0": 1}, num_spin_orbitals=1))

In [ ]:
a01 = anti_commutator(FermionicOp({"+_0": 1}, num_spin_orbitals=1), FermionicOp({"-_0": 1}, num_spin_orbitals=1))
a01.normal_order()

In [ ]:
anti_commutator(op1, op2)

In [ ]:
opx = FermionicOp({"+_0 -_1 +_1 -_3" : 1}, num_spin_orbitals=4)
op = FermionicOp(
    {
        "+_0 -_3": 1.0,
        "+_0 +_1 -_1 -_3": -1.0,
    },
    num_spin_orbitals=4,
)
opx.normal_order().equiv(op.normal_order())

In [ ]:
from qiskit.extensions import UnitaryGate, XGate, IGate, CXGate
from qiskit.quantum_info.operators import Operator, Pauli
from qiskit.visualization import array_to_latex
from qiskit.quantum_info import Statevector


import numpy as np

In [ ]:
sv0 = Statevector.from_label('0')
sv1 = Statevector.from_label('1')
X = Operator(Pauli('X'))
(sv0.evolve(X)).equiv(sv1)

In [ ]:
from qiskit.circuit.library import MCXGate, C3XGate
gate = MCXGate(4)

from qiskit import QuantumCircuit
circuit = QuantumCircuit(5)
circuit.append(gate, [0, 1, 4, 2, 3])
circuit.draw('mpl')

In [ ]:
Gat = C3XGate()
cir = QuantumCircuit(4)
cir.append(Gat, [0, 1, 3, 2])
cir.draw('mpl')

$
\newcommand{\HH}{\mathtt{H}}  
\newcommand{\ad}{a^\dagger}  
\newcommand{\kb}[2]{|#1\rangle\langle#2|} 
\newcommand{\proj}[1]{|#1\rangle\langle#1|}
\newcommand{\bk}[2]{\langle#1|#2\rangle}
\newcommand{\ket}[1]{|#1\rangle}
$

Opérateur qubit             |  Description
----------------------------|---------------------
$\mathtt{I}$$=\begin{pmatrix}1 & 0 \\ 0 &1 \end{pmatrix}$ |Identité
$\mathtt{Q}^-$$=\begin{pmatrix}0 & 1 \\ 1 & 0 \end{pmatrix}$$=\frac12(\mathtt{X} + i\mathtt{Y}) =\kb{1}{0}$ | Annihilation 
$\mathtt{Q}^+$$=\begin{pmatrix}0 & 0 \\ 1 & 0 \end{pmatrix}$$=\frac12(\mathtt{X} - i\mathtt{Y}) =\kb{0}{1}$ | Création
$\mathtt{Q}^+\mathtt{Q}^-$$=\begin{pmatrix}0 & 0 \\ 0 &1 \end{pmatrix}$$=\frac12(\mathtt{I} - \mathtt{Z}) =\proj{1}$ | Un nombre (particule)
$\mathtt{Q}^-\mathtt{Q}^+$$=\begin{pmatrix}0 & 0 \\ 0 &1 \end{pmatrix}$$=\frac12(\mathtt{I} + \mathtt{Z}) =\proj{0}$ | Zéro nombre (trou)

Les opérateurs qubits sont antisymétriques: $\{\mathtt{Q}^,\mathtt{Q}^-\}$$=\mathtt{Q}^+\mathtt{Q}^- + \mathtt{Q}^-\mathtt{Q}^+ = \mathtt{I}$.

In [ ]:
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *

In [ ]:
# Loading your IBM Q account(s)
provider = IBMQ.load_account()

device = provider.get_backend('ibmq_5_yorktown')

In [ ]:
qc=QuantumCircuit(3)
qc.ccx(0,1,2)
new_circuit = transpile(qc,backend=device)
new_circuit.draw(output='mpl')

In [ ]:
import pandas as pd

H2_data = pd.read_csv(f"Resultats/H2_results.csv")

H2_data

In [ ]:
import ipysheet
sheet = ipysheet.sheet(rows=3, columns=4)
cell1 = ipysheet.cell(0, 0, 'Hello')
cell2 = ipysheet.cell(2, 0, 'World')
cell_value = ipysheet.cell(2,2, 42.)
print(sheet)

In [ ]:
import ipywidgets as widgets
sheet = ipysheet.sheet(rows=3, columns=2, column_headers=False, row_headers=False)
cell_a = ipysheet.cell(0, 1, 1, label_left='a')
cell_b = ipysheet.cell(1, 1, 2, label_left='b')
cell_sum = ipysheet.cell(2, 1, 3, label_left='sum', read_only=True)

# create a slider linked to cell a
slider = widgets.FloatSlider(min=-10, max=10, description='a')
widgets.jslink((cell_a, 'value'), (slider, 'value'))

# changes in a or b should trigger this function
def calculate(change):
    cell_sum.value = cell_a.value + cell_b.value

cell_a.observe(calculate, 'value')
cell_b.observe(calculate, 'value')


widgets.VBox([sheet, slider])

## Qiskit_nature_pyscf version 0.6

In [ ]:
from pyscf import gto, scf, mcscf

import numpy as np

from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.algorithms.optimizers import SLSQP
from qiskit.primitives import Estimator
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD
from qiskit_nature.second_q.mappers import ParityMapper

from qiskit_nature_pyscf import QiskitSolver

mol = gto.M(atom="Li 0 0 0; H 0 0 1.6", basis="sto-3g")

h_f = scf.RHF(mol).run()

norb = 2
nalpha, nbeta = 1, 1
nelec = nalpha + nbeta

cas = mcscf.CASCI(h_f, norb, nelec)

# mapper = ParityMapper(num_particles=(nalpha, nbeta))
mapper = ParityMapper()

ansatz = UCCSD(
    norb,
    (nalpha, nbeta),
    mapper,
    initial_state=HartreeFock(
        norb,
        (nalpha, nbeta),
        mapper,
    ),
)

vqe = VQE(Estimator(), ansatz, SLSQP())
vqe.initial_point = np.zeros(ansatz.num_parameters)

algorithm = GroundStateEigensolver(mapper, vqe)

cas.fcisolver = QiskitSolver(algorithm)

cas.run()

In [ ]:
ansatz.num_qubits

$\newcommand{\mel}[3]{\langle #1|#2|#3\rangle} %Matrix element$


In [ ]:
H2_mol = gto.M(atom="H 0 0 0; H 0 0 .735", basis="sto-3g")

mf = scf.RHF (H2_mol).run ()

norb = mf.mo_coeff.shape[1]
nelec = H2_mol.nelec

print(norb)
print(nelec)

$\mel{\psi(\theta)}{\mathtt{H}}{\psi(\theta)}$

In [ ]:
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver

# Use PySCF, a classical computational chemistry software
# package, to compute the one-body and two-body integrals in
# electronic-orbital basis, necessary to form the Fermionic operator
driver = PySCFDriver(
    atom='H .0 .0 .0; H .0 .0 0.735',
    unit=DistanceUnit.ANGSTROM,
    basis='sto3g',
)
problem = driver.run()

# setup the qubit mapper
from qiskit_nature.second_q.mappers import ParityMapper

mapper = ParityMapper(num_particles=problem.num_particles)

# setup the classical optimizer for the VQE
from qiskit.algorithms.optimizers import L_BFGS_B

optimizer = L_BFGS_B()

# setup the estimator primitive for the VQE
from qiskit.primitives import Estimator

estimator = Estimator()

# setup the ansatz for VQE
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD

ansatz = UCCSD(
    problem.num_spatial_orbitals,
    problem.num_particles,
    mapper,
    initial_state=HartreeFock(
        problem.num_spatial_orbitals,
        problem.num_particles,
        mapper,
    ),
)

# set up our actual VQE instance
from qiskit.algorithms.minimum_eigensolvers import VQE

vqe = VQE(estimator, ansatz, optimizer)
# ensure that the optimizer starts in the all-zero state which corresponds to
# the Hartree-Fock starting point
vqe.initial_point = [0] * ansatz.num_parameters

# prepare the ground-state solver and run it
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

algorithm = GroundStateEigensolver(mapper, vqe)

electronic_structure_result = algorithm.solve(problem)
print(electronic_structure_result)

In [ ]:
ansatz.num_parameters


In [ ]:
'''
Response-theory calculation of the permanent dipole moment
'''

from pyscf import gto, scf, mcpdft
from pyscf.lib import logger

logger.TIMER_LEVEL = logger.INFO

# Energy calculation
h2co_xyz = '''C  0.534004  0.000000  0.000000
O -0.676110  0.000000  0.000000
H  1.102430  0.000000  0.920125
H  1.102430  0.000000 -0.920125'''
mol = gto.M (atom = h2co_xyz, basis = 'def2svp', symmetry = False,
             verbose = logger.INFO, output = '00-dipole_moment.log')
mf = scf.RHF (mol).run ()
mc = mcpdft.CASSCF (mf, 'tPBE', 6, 6)
mc.kernel ()

# Electric Dipole calculation
dipole = mc.dip_moment(unit='Debye')
print ("MC-PDFT electric dipole moment Debye")
print (" {:8.5f} {:8.5f} {:8.5f}".format (*dipole))
print ("Numerical MC-PDFT electric dipole moment from GAMESS [Debye]")
print (" 2.09361 0.00000 0.00000 ")

In [ ]:
#!/usr/bin/env python

'''
Custom solvent cavity
'''

import numpy
from pyscf import gto, qmmm, solvent

#
# Case 1. Cavity for dummy atoms with basis on the dummy atoms
#
mol = gto.M(atom='''
C        0.000000    0.000000             -0.542500
O        0.000000    0.000000              0.677500
H        0.000000    0.9353074360871938   -1.082500
H        0.000000   -0.9353074360871938   -1.082500
X-C      0.000000    0.000000             -1.5
X-O      0.000000    0.000000              1.6
            ''',
            verbose = 4)

sol = solvent.ddCOSMO(mol)
cavity_radii = sol.get_atomic_radii()

cavity_radii[4] = 3.0  # Bohr, for X-C
cavity_radii[5] = 2.5  # Bohr, for X-O
# Overwrite the get_atom_radii method to feed the custom cavity into the solvent model
sol.get_atomic_radii = lambda: cavity_radii

mf = mol.RHF().ddCOSMO(sol)
mf.run()


#
# Case 2. Cavity for dummy atoms (without basis)
#
mol = gto.M(atom='''
C        0.000000    0.000000             -0.542500
O        0.000000    0.000000              0.677500
H        0.000000    0.9353074360871938   -1.082500
H        0.000000   -0.9353074360871938   -1.082500
            ''',
            verbose = 4)

# Use a MM molecule to define cavity from dummy atoms.
# See also the example 22-with_qmmm.py
coords = numpy.array([
    [0, 0, -1.5],
    [0, 0, 1.6],
])
charges = numpy.array([0, 0])
mm_atoms = [('X', c) for c in coords]
mm_mol = qmmm.create_mm_mol(mm_atoms, charges)

# Make a giant system include both QM and MM particles
qmmm_mol = mol + mm_mol

# The solvent model is based on the giant system
sol = solvent.ddCOSMO(qmmm_mol)
cavity_radii = sol.get_atomic_radii()

# Custom cavity
cavity_radii[4] = 3.0  # Bohr
cavity_radii[5] = 2.5  # Bohr
# Overwrite the get_atom_radii method to feed the custom cavity into the solvent model
sol.get_atomic_radii = lambda: cavity_radii

mf = mol.RHF().QMMM(coords, charges)
mf = mf.ddCOSMO(sol)
mf.run()

In [ ]:
from pyscf import gto, scf, mcscf
from pyscf.mcscf import avas

mol = gto.M(
  atom="O 0 0 0.115; H 0 0.754 -0.459; H 0 -0.754 -0.459",
  basis="sto6g",
)

h_f = scf.RHF(mol).run()
# converged SCF energy = -75.6769190377958

# norb, nel, mo = avas.avas(h_f,['O 2p', 'H 1s','H 1s'])
nel = mol.nelec

# print(norb)
print(nel)

In [ ]:
LiH_mol = gto.M(atom="Li 0 0 0; H 0 0 1.6", basis="sto-3g")

LiH_h_f = scf.RHF(LiH_mol).run()
norb, nel, mo = avas.avas(h_f,['Li 2s', 'H 1s'])
# nel = mol.nelec

print(norb)
print(nel)
# print(mo)

In [ ]:
import numpy
from pyscf import gto, scf, mcscf

'''
Scan BeH2 molecule symmetric dissociation curve
Note the CI wave function might change symmetry in the scanning.  Adjust
fcisolver parameters to maintain the right symmetry.
'''

def run(i, dm0, mo0, ci0):
    x = i
    y = (2.54 - 0.46 * x)
    x = x * 0.529177249
    y = y * 0.529177249
    mol = gto.M(
        verbose = 0,
        atom = [
            ['Be',( 0., 0.    , 0.   )],
            ['H', ( x, -y    , 0.    )],
            ['H', ( x,  y    , 0.    )],],
        basis = '6-311G',
        symmetry = True)

    mf = scf.RHF(mol)
    ehf = mf.scf(dm0)

    mc = mcscf.CASSCF(mf, 2, 2)
    mc.fcisolver.davidson_only = True # force the CI solver stick on (A1)^2(B1)^0 configuration
    if mo0 is not None:
        mo0 = mcscf.project_init_guess(mc, mo0)

    emc = mc.mc1step(mo0, ci0)[0]

    print('%2.1f bohr, HF energy: %12.8f, CASSCF energy: %12.8f' % (i, ehf, emc))
    return mf, mc

dm0 = mo0 = ci = None
for i in reversed(numpy.arange(1.0, 4.1, .1)):
    mf, mc = run(i, dm0, mo0, ci)
    dm0 = mf.make_rdm1()
    mo_coeff = mc.mo_coeff
    ci = mc.ci

In [ ]:
from pandasgui import show

In [ ]:
from sklearn.datasets import load_breast_cancer
import pandas as pd

breast = load_breast_cancer()
breast_df = pd.DataFrame(breast.data, columns = breast.feature_names)
breast_df['target'] = breast.target
breast_df.head()

In [ ]:
show(breast_df)

In [ ]:
BS = (Operator(Pauli('I')) + 1.j*Operator(Pauli('X')))/np.sqrt(2)
array_to_latex(BS)

In [ ]:
BS = (Operator(IGate()) + 1.j*Operator(XGate()))/np.sqrt(2)
array_to_latex(BS)

In [ ]:
array_to_latex(Operator(CXGate()))

In [ ]:
BS = UnitaryGate(BS,'BS')
PBS = UnitaryGate(BS,'PBS')
# array_to_latex(BS)

In [ ]:
def BK(m):
    I = [[1, 0], [0, 1]]
    d = {}
    d[0] = [1]
    for k in range(0, m):
        B = np.kron(I,d[k])
        for l in range(2**k, 2**(k+1)):
            B[0,l] = 1
        d[k+1] = B
    return d


In [ ]:
from sympy.physics.quantum import *
import numpy as np

In [ ]:
psi = (Ket(00)+2*Ket(10)-2*Ket(11))/3
psi

In [ ]:
from ase import Atoms
from ase.build import molecule
from ase.visualize import view

In [ ]:
macro_ASE = Atoms('ONCHHHC', [(1.1280, 0.2091, 0.0000), 
                          (-1.1878, 0.1791, 0.0000), 
                          (0.0598, -0.3882, 0.0000),
                          (-1.3085, 1.1864, 0.0001),
                          (-2.0305, -0.3861, -0.0001),
                          (-0.0014, -1.4883, -0.0001),
                          (-0.1805, 1.3955, 0.0000)])

In [ ]:
view(macro_ASE, viewer='x3d')

In [ ]:
LiH_ASE = Atoms('LiH', [(0., 0., 0.), (0., 0., 1.5949)])
view(LiH_ASE, viewer='x3d')

In [ ]:
from qutip import *

In [ ]:
basis(2,0)

In [ ]:
plot_wigner(coherent(25,-2+2j)+coherent(25,2-2j))

In [ ]:
destroy(4)*create(4)*basis(4,3)

In [ ]:
(create(4)*destroy(4)+1)*basis(4,3)

In [ ]:
! pip install jupyter_nbextensions_configurator

In [ ]:
! jupyter nbextensions_configurator enable --user